# Introduction to the Model Context Protocol (MCP)
**For Computational Linguistics & NLP Students**

## 1. What is MCP? (5 minutes)

### The Problem: The M×N Integration Nightmare

Imagine you want Claude, ChatGPT, and a local Llama model to all access your annotated corpus. Traditionally, you'd need to write **3 separate integrations** - one for each model's specific API.

Now imagine you add 2 more tools (a sentiment analyzer and a dependency parser). Now you need **3 × 3 = 9 integrations**!

### The Solution: MCP as a Universal Standard

**The Model Context Protocol (MCP)** is like **USB-C for AI**:
- **USB-C**: One port works with any device (mouse, keyboard, drive)
- **MCP**: One server works with any AI client (Claude, Cursor, custom apps)

Write your corpus server **once**, and any MCP-compatible AI can use it.

<img src="https://mintcdn.com/mcp/bEUxYpZqie0DsluH/images/mcp-simple-diagram.png?fit=max&auto=format&n=bEUxYpZqie0DsluH&q=85&s=35268aa0ad50b8c385913810e7604550" alt="MCP Diagram" width="800"/>

<img src="https://cdn.hashnode.com/res/hashnode/image/upload/v1761561683902/6b0f2299-041e-4ef4-a6ce-726899c52fbf.png" alt="MCP USB-C Analogy" width="600"/>

### Three Simple Primitives

MCP has just three building blocks:

1. **Resources**: Data the AI can read (files, database entries, API responses)
2. **Tools**: Functions the AI can execute (calculate statistics, run analyses)
3. **Prompts**: Templates that guide the AI's use of your server

---

## 2. MCP vs. REST: A Different Philosophy (5 minutes)

As students familiar with REST APIs (like FastAPI), you might wonder: "Isn't this just another API?"

**No.** MCP represents a fundamental paradigm shift.

### The Core Difference

| Dimension | REST API | MCP |
|-----------|----------|-----|
| **Design Philosophy** | **Resource-oriented** (nouns)<br/>"What data exists?" | **Action-oriented** (verbs)<br/>"What can I do?" |
| **Mindset** | **Unopinionated**<br/>"Here is raw data; you figure out how to use it." | **Goal-Oriented**<br/>"Here are specific tools to help you achieve a task." |
| **Primary User** | Human developers reading docs | AI agents discovering capabilities |
| **Discovery** | Manual: Read OpenAPI/Swagger | Automatic: Server advertises tools |
| **Logic Location** | **Client-Side**<br/>The Agent must loop/filter/sort. | **Server-Side**<br/>The Server handles the complexity. |
| **Semantics** | Low-level CRUD operations | High-level actions |

### Example: The Difference in Practice

**Task:** Archive all blog posts from 2023

**REST approach (what agent must do):**
```python
# The Agent has to:
# 1. Know the endpoint URL schema
# 2. Handle pagination (loops)
# 3. Handle JSON formatting manually

page = 1
while True:
    data = requests.get(f"/api/blogs?year=2023&page={page}").json()
    if not data: break
    for post in data:
         # Agent must know specific field names ('id', 'status')
         requests.put(f"/api/blogs/{post['id']}", json={"status": "archived"})
    page += 1
```
Risk: The Agent might hallucinate the loop logic or mess up the ID format.

**MCP Approach (The "Contractor" Method - Turnkey Service): The Server anticipates the goal and provides a high-level tool.**
```python
# One high-level action
# The Agent simply calls the tool:
await session.call_tool("archive_old_posts", arguments={"year": 2023})
```
✓ Single operation, atomic, server handles complexity.

### Why This Matters for Linguistics

When building NLP tools, you want agents to think:
- "Annotate this text with POS tags" ✓

Not:
- "GET the text, POST it to the tagger, PUT the results back" ✗

MCP lets you expose **linguistic operations** rather than **data CRUD**.

---

## 3. How MCP Works (Simple Version)

### The Architecture (High-Level)

```
┌─────────────────┐
│   AI Client     │  (Claude, Cursor, etc.)
│   (The Host)    │
└────────┬────────┘
         │
         │ JSON-RPC messages
         │ over stdio/HTTP/SSE
         │
┌────────▼────────┐
│   MCP Server    │  (Your linguistic tools)
│                 │
│  • Tools        │  calculate_ttr()
│  • Resources    │  corpus://text/123
│  • Prompts      │  analyze_complexity()
└─────────────────┘
```

### The Contrast: Connecting to Tools

**Option A: The REST Handshake (Manual Reality)**

1.  **Connection**: Developer finds API documentation URL, hardcodes API keys, and initializes a generic HTTP client.
2.  **Discovery (The Hard Part)**: Developer **manually reads** 50 pages of Swagger/OpenAPI documentation, selects relevant endpoints, and writes a custom "System Prompt" to explain how `GET /corpus` works to the AI.
3.  **Usage**: AI outputs raw JSON; Developer writes `try/except` blocks to parse it and make the HTTP call.
4.  **Shutdown**: No protocol. Stateless.

**Option B: The MCP Handshake (Automated Reality)**

1.  **Connection**: Client connects to server (via command-line process).
2.  **Discovery**: Server **automatically** tells client: "I can do X, Y, Z."
3.  **Usage**: Client calls tools as needed directly.
4.  **Shutdown**: Clean disconnect.

**Key insight**: The AI doesn't need to read documentation—the server describes itself\!

---

## 4. Demo: A Linguistic Analysis Server (15 minutes)

Let's build a simple MCP server for corpus analysis using `fastmcp` (think "FastAPI for MCP").

In [ ]:
# Install if needed
# %pip install fastmcp

In [ ]:
from typing import Any

from fastmcp import FastMCP

# Initialize server (like FastAPI's app = FastAPI())
mcp = FastMCP("LinguistHelper")

# Mock corpus (in reality, this could be a database)
CORPUS = {
    "en_sample_01": "The quick brown fox jumps over the lazy dog.",
    "en_sample_02": "To be, or not to be, that is the question.",
    "zh_sample_01": "學而不思則罔,思而不學則殆。",
}

# ============================================================================
# RESOURCES: Give AI access to data
# ============================================================================


@mcp.resource("corpus://list")
def list_texts() -> str:
    """List all available text IDs in the corpus."""
    return "\n".join(CORPUS.keys())


@mcp.resource("corpus://{text_id}")
def get_text(text_id: str) -> str:
    """Retrieve text content by ID."""
    if text_id not in CORPUS:
        available = ", ".join(CORPUS.keys())
        return f"Error: Text '{text_id}' not found. Available: {available}"
    return CORPUS[text_id]


# ============================================================================
# TOOLS: Give AI computational capabilities
# ============================================================================


@mcp.tool()
def calculate_ttr(text: str) -> float:
    """
    Calculate Type-Token Ratio (TTR) of a text.
    TTR = (Unique Words / Total Words)
    Higher values indicate greater lexical diversity.

    Example: "the cat sat on the mat" -> 5/6 = 0.83
    """
    if not text:
        return 0.0

    tokens = text.lower().split()  # Simple tokenization
    types = set(tokens)  # Unique words

    return len(types) / len(tokens) if tokens else 0.0


@mcp.tool()
def count_words(text: str) -> int:
    """Count total words in text."""
    return len(text.split())


@mcp.tool()
def find_longest_word(text: str) -> str:
    """Find the longest word in the text."""
    words = text.split()
    return max(words, key=len) if words else ""


# ============================================================================
# PROMPTS: Pre-built workflows for common tasks
# ============================================================================


@mcp.prompt()
def analyze_complexity(text_id: str) -> str:
    """Generate a prompt for analyzing text complexity."""
    return f"""
Please analyze the linguistic complexity of corpus://{text_id}.

Steps:
1. First, read the text using the corpus://{text_id} resource
2. Calculate its Type-Token Ratio using the calculate_ttr tool
3. Find the longest word using the find_longest_word tool
4. Provide a brief assessment of lexical complexity

A TTR above 0.7 typically indicates rich vocabulary.
"""


print("✓ MCP Server 'LinguistHelper' initialized!")
print(f"  Resources: {len(mcp._resource_manager._resources)}")
print(f"  Tools: {len(mcp._tool_manager._tools)}")

### What Just Happened?

We created three types of capabilities:

1. **Resources** (`@mcp.resource`):
   - `corpus://list` - Lists available texts
   - `corpus://{text_id}` - Retrieves specific text
   - Think: "Things the AI can read"

2. **Tools** (`@mcp.tool`):
   - `calculate_ttr()` - Computes lexical diversity
   - `count_words()` - Simple word count
   - `find_longest_word()` - Finds longest token
   - Think: "Things the AI can do"

3. **Prompts** (`@mcp.prompt`):
   - `analyze_complexity()` - Pre-built workflow
   - Think: "Recipes for common tasks"

### How an AI Uses This

**User asks:** "Is en_sample_01 lexically diverse?"

**AI's reasoning:**
1. "I should read the text first" → Calls `corpus://en_sample_01`
2. "Now calculate diversity" → Calls `calculate_ttr(text)`
3. "Interpret result" → Returns: "Yes, TTR of 0.89 indicates high diversity"

The AI **automatically discovers** these capabilities and chains them together!

### Running the Server

To actually deploy this:

```bash
# Save the code above to linguistic_server.py
# Then run:
fastmcp run linguistic_server.py

# Or install in Claude Desktop:
# Add to claude_desktop_config.json:
{
  "mcpServers": {
    "linguistic-tools": {
      "command": "python",
      "args": ["-m", "fastmcp", "run", "linguistic_server.py"]
    }
  }
}
```

Now Claude can use your linguistic tools in any conversation!

---

## 5. Key Design Principles (3 minutes)

### Principle 1: Think Actions, Not CRUD

❌ **Bad** (REST-style):
```python
@mcp.tool()
def get_text_data(id: str) -> dict:
    """Get text data"""
    return database.get(id)

@mcp.tool()
def update_text_data(id: str, data: dict):
    """Update text data"""
    database.update(id, data)
```

✅ **Good** (Action-oriented):
```python
@mcp.tool()
def annotate_with_pos_tags(text_id: str) -> str:
    """Add part-of-speech tags to a corpus text."""
    text = corpus.get(text_id)
    tagged = pos_tagger.tag(text)
    corpus.save_annotation(text_id, "pos", tagged)
    return tagged
```

### Principle 2: Encapsulate Domain Logic

Don't make the AI orchestrate multiple steps. Give it high-level operations:

```python
@mcp.tool()
def full_linguistic_analysis(text_id: str) -> dict:
    """Run complete analysis pipeline on a text."""
    text = corpus.get(text_id)
    
    return {
        "ttr": calculate_ttr(text),
        "word_count": count_words(text),
        "longest_word": find_longest_word(text),
        "pos_tags": pos_tag(text),
        "readability": flesch_reading_ease(text)
    }
```

### Principle 3: Security First

```python
import os

@mcp.tool()
def query_proprietary_corpus(query: str) -> str:
    """Search our proprietary database."""
    # API key stays server-side - AI never sees it!
    api_key = os.environ["CORPUS_API_KEY"]
    return search_api(query, api_key)
```

The AI can search, but can't leak credentials.

---

## 6. Why This Matters for Your Research (2 minutes)

### Use Cases in Computational Linguistics

**1. Corpus Access**
```python
@mcp.resource("wals://feature/{feature_id}")
def get_wals_feature(feature_id: str) -> str:
    """Access World Atlas of Language Structures."""
    # Now any AI can query typological databases!
```

**2. Annotation Pipelines**
```python
@mcp.tool()
def annotate_universal_dependencies(text: str, language: str) -> str:
    """Parse text and return CoNLL-U format."""
    # Give LLMs access to linguistic parsers
```

**3. Fieldwork Tools**
```python
@mcp.tool()
def generate_elicitation_prompt(phenomenon: str) -> str:
    """Create prompts for linguistic elicitation."""
    # AI-assisted fieldwork!
```

### The Big Picture

**Before MCP:**
- Every AI tool needed custom integration
- Researchers duplicated effort
- Tools were fragmented

**With MCP:**
- Write linguistic tools **once**
- Works with **any** MCP-compatible AI
- Build a **reusable ecosystem** of computational linguistics tools

---

## Summary: Key Takeaways

1. **MCP is USB-C for AI** - one protocol, works everywhere

2. **MCP ≠ REST** - It's action-oriented (verbs) not resource-oriented (nouns)

3. **Three primitives**: Resources (data), Tools (functions), Prompts (workflows)

4. **Design for actions** - Think "annotate text" not "GET/PUT text data"

5. **Security built-in** - Credentials stay server-side, AI can't leak secrets

6. **Perfect for linguistics** - Share corpus access, annotation tools, and analysis pipelines

### Next Steps

**Try it yourself:**
1. Install fastmcp: `pip install fastmcp`
2. Save the demo code to `linguistic_server.py`
3. Run: `fastmcp run linguistic_server.py`
4. Test with Claude Desktop or build your own client

**Extend it:**
- Add tools for your specific research (dependency parsing, NER, etc.)
- Connect to real corpora (Universal Dependencies, your own datasets)
- Build prompts for common linguistic analysis tasks

**Resources:**
- Official docs: https://modelcontextprotocol.io/
- fastmcp: https://github.com/jlowin/fastmcp
- Examples: https://github.com/modelcontextprotocol/servers

---

## References and Further Reading

### Primary Sources

1. **Lee, H. (2025).** "MCP is not REST API." *Personal Blog*.  
   https://leehanchung.github.io/blogs/2025/05/17/mcp-is-not-rest-api/  
   *Core reading on action-oriented vs resource-oriented design and why wrapping REST as MCP is problematic.*

2. **Shivanandhan, M. (2025).** "MCP vs APIs: What's the Real Difference?" *freeCodeCamp*.  
   https://www.freecodecamp.org/news/mcp-vs-apis-whats-the-real-difference/  
   *Accessible introduction to MCP with focus on security and safety benefits.*

### Official Documentation

3. **Anthropic.** "Model Context Protocol - Introduction."  
   https://modelcontextprotocol.io/introduction

4. **Anthropic.** "Model Context Protocol - Transports."  
   https://modelcontextprotocol.io/docs/concepts/transports  
   *Technical details on JSON-RPC 2.0 and transport mechanisms.*

5. **Anthropic.** "Model Context Protocol - Resources."  
   https://modelcontextprotocol.io/docs/concepts/resources

6. **Anthropic.** "Model Context Protocol - Tools."  
   https://modelcontextprotocol.io/docs/concepts/tools

### Technical Deep Dives

7. **Loganathan, P. (2025).** "MCP - Protocol Mechanics and Architecture." *Personal Blog*.  
   https://pradeepl.com/blog/model-context-protocol/mcp-protocol-mechanics-and-architecture/  
   *Detailed explanation of the three-layer architecture and connection lifecycle.*

8. **Avila, D. (2025).** "Why Model Context Protocol uses JSON-RPC." *Medium*.  
   https://medium.com/@dan.avila7/why-model-context-protocol-uses-json-rpc-64d466112338  
   *Rationale behind JSON-RPC 2.0 choice and transport flexibility.*

9. **IBM.** "What is Model Context Protocol (MCP)?"  
   https://www.ibm.com/think/topics/model-context-protocol  
   *Enterprise perspective on MCP architecture and use cases.*

10. **Composio.** "What is Model Context Protocol (MCP): Explained."  
    https://composio.dev/blog/what-is-model-context-protocol-mcp-explained  
    *Overview of base protocol, lifecycle, and practical examples.*

### Related Protocols and Context

11. **Microsoft.** "Language Server Protocol - Overview."  
    https://microsoft.github.io/language-server-protocol/overviews/lsp/overview/  
    *MCP's inspiration - understanding LSP helps understand MCP's design philosophy.*

12. **JSON-RPC Working Group.** "JSON-RPC 2.0 Specification."  
    https://www.jsonrpc.org/specification  
    *The underlying messaging format used by MCP.*

### Implementation Libraries

13. **Lowin, J.** "fastmcp - FastAPI for MCP servers."  
    https://github.com/jlowin/fastmcp  
    *Python library used in this notebook's examples.*

14. **Anthropic.** "MCP TypeScript/JavaScript SDK."  
    https://github.com/modelcontextprotocol/typescript-sdk

15. **Anthropic.** "MCP Python SDK."  
    https://github.com/modelcontextprotocol/python-sdk

### Community Examples

16. **Model Context Protocol Servers Repository.**  
    https://github.com/modelcontextprotocol/servers  
    *Official collection of example MCP servers (filesystem, GitHub, PostgreSQL, etc.)*

---

### Recommended Reading Order

**For beginners:**
1. Start with freeCodeCamp article [2] for accessible overview
2. Read Lee Hanchung's article [1] for conceptual understanding
3. Try the official introduction [3]
4. Experiment with fastmcp [13]

**For implementers:**
1. Official documentation [3-6]
2. Technical architecture deep dive [7]
3. Implementation SDKs [14-15]
4. Community examples [16]

**For researchers:**
1. Lee Hanchung on design philosophy [1]
2. Protocol mechanics [7]
3. LSP comparison [11] (to understand the inspiration)